In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-01-24 09:29:07--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   840KB/s    in 16s     

2023-01-24 09:29:23 (1.52 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [78]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [79]:
df_books.shape

(271379, 3)

In [80]:
df_ratings.shape

(1149780, 3)

In [5]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [6]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [ ]:
# add your code here - consider creating a new cell for each section of code

In [81]:
df_ratings['user'].nunique()

105283

In [110]:
# remove users with less than 200 ratings and books with less than 100
df_ratings_2 = df_ratings.groupby('user').filter(lambda x: len(x) >= 200)

df_ratings_3 = df_ratings_2.groupby('isbn').filter(lambda x: len(x) >= 100)

In [111]:
df_ratings_3.shape

(13793, 3)

In [112]:
df_ratings_3['isbn'].nunique()

100

In [113]:
df_ratings_3['user'].nunique()

857

In [106]:
# try same thing with a mask
user_mask = df_ratings.groupby('user')['rating'].transform('size') >= 200
book_mask = df_ratings.groupby('isbn')['rating'].transform('size') >= 100
df_ratings_4 = df_ratings[user_mask & book_mask]

In [109]:
df_ratings_4.shape

(49781, 3)

In [107]:
df_ratings_4['isbn'].nunique()

731

In [108]:
df_ratings_4['user'].nunique()

888

In [114]:
# shape the data into a matrix with rows for books and columns for users
df_pivot = df_ratings_4.pivot(index='isbn', columns='user', values='rating').fillna(0)
df_pivot.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
isbn,,,,,,,,,,,,,,,,,,,,,
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
0060008032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060096195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006016848X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060173289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
# get row # of an isbn in the matrix
df_pivot.index.get_loc('0060930535')

2

In [116]:
# Create a sparse matrix
ratings_matrix = csr_matrix(df_pivot.values)

In [117]:
# create the model
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5)

In [118]:
# fit the model
model.fit(ratings_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [119]:
def get_isbn(book_name):
    return df_books[df_books['title'] == book_name].iloc[0]['isbn']

def get_index(isbn):
    return df_pivot.index.get_loc(isbn)

def get_name(index):
    isbn = df_pivot.index[index]
    return df_books[df_books['isbn'] == isbn]['title'].item()

In [121]:
isbn = get_isbn("Where the Heart Is (Oprah's Book Club (Paperback))")
idx = get_index(isbn)
print(f'isbn: {isbn}, index: {idx}')

isbn: 0446672211, index: 435


In [122]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
    isbn = get_isbn(book)
    print(f'Getting recommendations for {book} (isbn: {isbn})')
    # get index of the row in our matrix
    index = get_index(isbn)
    # get the row in the matrix
    matrix_row = ratings_matrix[index]
    
    # get recommendations
    distances, indices = model.kneighbors(matrix_row, n_neighbors=6)
    
    print(distances)
    print(indices)
    # loop through results and assemble in the format ["book title", distance]
    recommended_books = []
    return recommended_books

In [123]:
get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
#get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")

Getting recommendations for Where the Heart Is (Oprah's Book Club (Paperback)) (isbn: 0446672211)
[[0.         0.7234864  0.7677075  0.7699411  0.77085835 0.8016211 ]]
[[435 108  27 182 120 625]]


[]

In [129]:
get_name(625)

"I'll Be Seeing You"

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()